In [ ]:
import pandas as pd
import numpy as np
np.random.seed(2022)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
#for embedding


ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
df = pd.read_csv('/Users/macbook/Documents/my_nlp_project/data/personal_transaction.csv')

In [ ]:

df = df[['Description', 'Category']]

In [ ]:
df.head()

,Description,Category
0,Amazon,Shopping
1,Mortgage Payment,Mortgage & Rent
2,Thai Restaurant,Restaurants
3,Credit Card Payment,Credit Card Payment
4,Netflix,Movies & DVDs


In [ ]:
import spacy
import string
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'nevertheless', 'me', 'meanwhile', 'anything', 'twenty', 'fifteen', 'was', 'nobody', 'first', 'again', 'always', 'else', 'ten', 'which', 'twelve', 'together', 'then', 'himself', 'beforehand', 'take', 'become', 'also', 'everyone', 'by', 'often', 'whence', 'against', 'i', 'third', 'under', 'or', 'should', 'otherwise', 'wherever', 'it', 'no', 'thru', 'will', 'many', 'whose', 'indeed', 'may', 'he', "'d", 'moreover', 'thus', 'although', 'why', 'mostly', 'anywhere', 'anyway', 'several', 'towards', 'about', 'seem', 'quite', 'of', 'unless', 'they', 'amount', 'sixty', 'her', 'ca', 'a', '‘m', 'even', 'being', 'such', 'either', 'others', 'an', 'might', 'per', 'last', 'when', 'empty', 'forty', 'therefore', 'elsewhere', 'across', 'noone', 'with', 'formerly', 'is', 'through', 'your', 'hundred', 'within', 'were', 'once', 'than', 'therein', 'whereafter', 'where', 'yours', 'if', 'hence', 'above', 'yet', 'hereafter', 'very', 'ourselves', 'other', 'front', 'eight', 'made', 'whereby', 'that', 'n‘t', 'are

In [ ]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
from sklearn.preprocessing import LabelEncoder
# Initialiser l'encodeur
label_encoder = LabelEncoder()

# Appliquer l'encodage
df['category_encoded'] = label_encoder.fit_transform(df['Category'])

In [ ]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)



    print(doc)
    print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    
    sentence = " ".join(mytokens)
    # return preprocessed list of tokens
    return sentence

In [ ]:
print(spacy_tokenizer("I am Eating an aple"))


I am Eating an aple
<class 'spacy.tokens.doc.Doc'>
eat aple


In [ ]:
df['tokenize'] = df['Description'].apply(spacy_tokenizer)

Amazon
<class 'spacy.tokens.doc.Doc'>
Mortgage Payment
<class 'spacy.tokens.doc.Doc'>
Thai Restaurant
<class 'spacy.tokens.doc.Doc'>
Credit Card Payment
<class 'spacy.tokens.doc.Doc'>
Netflix
<class 'spacy.tokens.doc.Doc'>
American Tavern
<class 'spacy.tokens.doc.Doc'>
Hardware Store
<class 'spacy.tokens.doc.Doc'>
Gas Company
<class 'spacy.tokens.doc.Doc'>
Hardware Store
<class 'spacy.tokens.doc.Doc'>
Spotify
<class 'spacy.tokens.doc.Doc'>
Phone Company
<class 'spacy.tokens.doc.Doc'>
Shell
<class 'spacy.tokens.doc.Doc'>
Grocery Store
<class 'spacy.tokens.doc.Doc'>
Biweekly Paycheck
<class 'spacy.tokens.doc.Doc'>
Pizza Place
<class 'spacy.tokens.doc.Doc'>
Amazon
<class 'spacy.tokens.doc.Doc'>
Grocery Store
<class 'spacy.tokens.doc.Doc'>
American Tavern
<class 'spacy.tokens.doc.Doc'>
City Water Charges
<class 'spacy.tokens.doc.Doc'>
Power Company
<class 'spacy.tokens.doc.Doc'>
Biweekly Paycheck
<class 'spacy.tokens.doc.Doc'>
Amazon
<class 'spacy.tokens.doc.Doc'>
Credit Card Payment
<clas

In [ ]:
df.head()

,Description,Category,category_encoded,tokenize
0,Amazon,Shopping,19,amazon
1,Mortgage Payment,Mortgage & Rent,14,mortgage payment
2,Thai Restaurant,Restaurants,18,thai restaurant
3,Credit Card Payment,Credit Card Payment,3,credit card payment
4,Netflix,Movies & DVDs,15,netflix


In [ ]:
df['embeddings'] = df['tokenize'].apply(model.encode)

In [ ]:
df.head()

,Description,Category,category_encoded,tokenize,embeddings
0,Amazon,Shopping,19,amazon,"[-0.023181925, -0.026144259, -0.06706474, 0.03..."
1,Mortgage Payment,Mortgage & Rent,14,mortgage payment,"[-0.005457, 0.028130258, -0.026588269, -0.0341..."
2,Thai Restaurant,Restaurants,18,thai restaurant,"[-0.015605605, 0.019369751, -0.0051350724, 0.0..."
3,Credit Card Payment,Credit Card Payment,3,credit card payment,"[-0.004055636, 0.05979043, -0.017683476, 0.024..."
4,Netflix,Movies & DVDs,15,netflix,"[-0.047873974, -0.15620917, -0.048543725, -0.0..."


In [ ]:
#df = df[df['category_encoded'].map(df['category_encoded'].value_counts()) > 1]  # Supprime les classes avec 1 seul échantillon
X = df['embeddings'].to_list()
y = df['category_encoded'].to_list()


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [ ]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
# Calcul de la précision pour chaque classe
print("Logistic Regression Precision per class:", metrics.precision_score(y_test, predicted, average=None))
print("Logistic Regression Recall per class:", metrics.recall_score(y_test, predicted, average=None))
     


Logistic Regression Accuracy: 0.9691358024691358
Logistic Regression Precision per class: [1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.33333333 1.         1.         0.86666667 0.90909091 0.
 1.        ]
Logistic Regression Recall per class: [0.75       1.         1.         1.         1.         1.
 0.95833333 1.         1.         1.         1.         1.
 0.5        1.         1.         1.         1.         0.
 1.        ]


/Users/macbook/Documents/zz2_project/zz2_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

# Crée ton modèle RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=500, random_state=42)



# Utilisation de la validation croisée avec 5 plis (folds)
cv_scores = cross_val_score(classifier, X, y, cv=5, scoring='accuracy')  # Tu peux ajuster le scoring (par exemple, 'accuracy', 'precision', etc.)

# Affichage des résultats
print(f"Cross-validation Accuracy Scores: {cv_scores}")
print(f"Mean Accuracy: {np.mean(cv_scores)}")
print(f"Standard Deviation: {np.std(cv_scores)}")

classifier.fit(X_train,y_train)
# Save the model and label encoder
joblib.dump(classifier, 'model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

/Users/macbook/Documents/zz2_project/zz2_env/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation Accuracy Scores: [0.95679012 0.96273292 0.95652174 0.98757764 0.96273292]
Mean Accuracy: 0.9652710681696188
Standard Deviation: 0.011479936562777146


['label_encoder.pkl']